# django中的悲观锁和乐观锁

悲观锁是真正的锁，一般是和事务一起使用。



In [ ]:
class MyView(View):
    
    @transaction.atomic
    def post(self, request):
        # select * from 表名 where id=1 for update;  
        # for update 就表示锁,只有获取到锁才会执行查询,否则阻塞等待。
        obj = 模型类名.objects.select_for_update().get(id=1)
        
        # 等事务提交后，会自动释放锁。
        
        return HttpResponse('ok')


**乐观锁其实并不是锁。通过SQL的where子句中的条件是否满足来判断是否满足更新条件来更新数据库，通过受影响行数判断是否更新成功，如果更新失败可以再次进行尝试，如果多次尝试失败就返回更新失败的结果。**

使用乐观锁时，必须设置数据库的隔离级别是Read Committed(可以读到其他线程已提交的数据)。如果隔离级别是Repeatable Read(可重复读，读到的数据都是开启事务时刻的数据，即使其他线程提交更新数据，该线程读取的数据也是之前读到的数据)，乐观锁如果第一次尝试失败,那么不管尝试多少次都会失败。(Mysql数据库的默认隔离级别是Repeatable Read，需要修改成Read Committed)。


In [ ]:
from django.db import transaction

class MyView(View):
    
    @transaction.atomic
    def post(self, request):
        '''订单创建'''
        count = 3   # 订购3件商品
        
        # 设置事务保存点
        s1 = transaction.savepoint()
        
        # 乐观锁，最多尝试5次
        for i in range(5):
            # 查询商品的信息(库存)
            try:
                sku = GoodsSKU.objects.get(id=1)
            except:
                # 商品不存在
                transaction.savepoint_rollback(s1)
                return JsonResponse({'res': 1, 'errmsg': '商品不存在'})
 
            # 判断商品的库存, 这里不严谨，因为库存是可以增多的
            if count > sku.stock:
                transaction.savepoint_rollback(s1)
                return JsonResponse({'res': 2, 'errmsg': '商品库存不足'})
 
            # 更新商品的库存和销量
            orgin_stock = sku.stock   # 原库存 (数据库隔离级别必须是Read Committed；如果是Repeatable Read,那么多次尝试读取的原库存都是一样的,读不到其他线程提交更新后的数据。)
            new_stock = orgin_stock - count   # 更新后的库存
            new_sales = sku.sales + count   # 更新后的销量
 
            # update 商品表 set stock=new_stock, sales=new_sales where id=1 and stock = orgin_stock
            # 通过where子句中的条件判断库存是否进行了修改。(并发，乐观锁)
            # 返回受影响的行数
            res = GoodsSKU.objects.filter(id=1, stock=orgin_stock).update(stock=new_stock, sales=new_sales)
            if res == 0:  # 如果修改失败
                if i == 4:
                    # 如果尝试5次都失败
                    transaction.savepoint_rollback(s1)
                    return JsonResponse({'res': 3, 'errmsg': '下单失败'})
                continue  # 再次尝试
 
            # 否则更新成功
            # 跳出尝试循环
            break
 
 
        # 提交事务
        transaction.savepoint_commit(s1)
 
        # 返回应答
        return JsonResponse({'res': 4, 'message': '创建成功'})


# 事务

## 针对每次HTTP请求的事务交易

Django1.5和1.6版本都允许用户操作请求事务模式。在这种模式下，Django会自动在事务中，处理你的视图函数。如果视图函数抛出异常，Django会自动回滚事务；否则Django会提交事务。

为了实现这个功能，你需要在你想要有此功能的数据库的配置中，设置“ATOMIC_REQUEST”为真。所以在我们的“settings.py”需要有如下设置： 

```
DATABASES = {
  'default': {
    'ENGINE': 'django.db.backends.sqlite3',
    'NAME': os.path.join(SITE_ROOT, 'test.db'),
    'ATOMIC_REQUEST': True,
  }
}

```

## 异常捕获在atomic里

In [ ]:
with transaction.atomic():
    try:
        Province.objects.create(name="北京")
        int('a')
        Province.objects.create(name="上海")
    except:
        print('=======')

当退出一个atomic块时，Django会查看它是正常退出还是有异常来确定是提交还是回滚。如果你捕获并处理一个atomic块内的异常，atomic认为没有异常。不会执行它特有的回滚功能，也就是说认为代码块里的东西都是正常的。第一句Province.objects.create(name="北京")会执行成功，执行到int('a')执行失败下面的Province.objects.create(name="上海")不会继续执行。所以数据库会多一条"北京"的记录, 这种情形可能不是我们想要的。

## 异常捕获在atomic外



In [ ]:
try:
    with transaction.atomic():
        Province.objects.create(name="北京")
        int('a')
        Province.objects.create(name="上海")
except:
    print('======')

此时atomic代码块执行失败，这是atomic看的见的异常，所以会执行回滚，此时数据库没有新增记录。这可能是我们经常期望看到的

## atomic的嵌套



In [ ]:
with transaction.atomic():
    obj = Province.objects.create(name="北京")
    try:
        with transaction.atomic():
            Province.objects.create(name="北京2", active=3)
    except Exception as e:
        print(e)

In [ ]:
最终数据库只会多一条name: 北京 的记录。

## 保存点

尽管事务是有原子性的，但还是能够打散为多个"保存点"——可看作是"部分事务"。

例如，你有个事务包含了4条SQL语句，你可以在第二个SQL之后创建一个保存点。一旦保存点创建成功，就算第三条或第四条SQL执行失败，你仍旧能够做一个部分回滚，忽视后面两条SQL，仅保留前面两条。

基本上，这就像是提供了一个切割的能力：一个普通的事务能够被切分为多个更“轻量”的事务，然后能进行部分回滚或部分提交。但一定要注意，小心整个事务被无端回滚掉（例如由于抛出了IntegrityError异常但却没有捕抓，那所有的保存点都会因此被回滚掉的).



In [ ]:
with transaction.atomic():
    obj = Province.objects.create(name="北京")
    sid1 = transaction.savepoint()
    obj.name = '北京1'
    obj.save()
    # transaction.savepoint_rollback(sid1)
    transaction.savepoint_commit(sid1)


执行 savepoint_rollback(sid1) 最终数据库新增一条记录 name: 北京；执行savepoint_commit，会把保存点到savepoint_commit这句代码之间的都提交，最新数据库新增的一条记录 name: 北京1 

## 几个注意点

### transaction事务内不执行数据库的commit操作

例如在事务当前启动celery异步任务, 无法获取未提交的改动.在使用transaction当中, Model.save()都不做commit,因此如果在transaction当中设置异步任务,使用get()查询数据库,将看不到对象在事务当中的改变.这也是实现”可重复读”的事务隔离级别,即同一个事务里面的多次查询都应该保持结果不变.

In [ ]:
def example_view(request):
    with transaction.atomic():
        change_obj() # 修改对象变量
        obj.save()
        async_task.delay(obj.id)

def async_task(obj_id):
    obj = Model.objects.get(pk=obj_id)
    read_the_obj() # 读取对象信息

### transaction只对数据库层的操作进行事务管理,不能理解为python操作的事务管理

即使事务代码块发生了DataError,事务回滚,也仅是数据库层面的回滚,针对python的操作依然已完成.甚至是对Model.Object进行的操作会也会存在变量当中. 

In [ ]:
def example_view(request):
    obj.changed = False
    with transaction.atomic():
        obj.changed = True
        change_obj() # 修改对象其他变量
        obj.save()
        raise DataError
    # True
    print("obj.changed = ",obj.changed)


发生Dataerror异常的回滚仅在数据库层面操作,因此不可以根据model object的属性值判断是否正确完成了事务. 

另外,虽然Django对数据库层面以ORM完成了很具体的抽象,但应该要清楚地意识到我们操作的model object和数据库内容本质不同,DJANGO只在查询和提交时进行数据库操作.
